In [6]:
import TWIML_codenames
import numpy as np
import random
from datetime import datetime

np.random.seed(42)
wordlist = [line.strip() for line in open('wordlist.txt', 'r').readlines()][:25]

playerlist=[]
for i in range(4):
    playerlist.append(TWIML_codenames.Player(1001+i))

# first split the players into teams:
team1 = playerlist[:2]
team2 = playerlist[2:]

In [7]:
# gameboard = TWIML_codenames.Gameboard(wordlist)
gameboard = TWIML_codenames.Gameboard(wordlist)
# then create the game:
my_game = TWIML_codenames.Game(gameboard = gameboard, team1 = team1, team2 = team2)
print(my_game.gameboard.boardwords)
print(my_game.gameboard.boardkey)

[['body' 'kilogram' 'sick' 'team' 'hook']
 ['clean' 'barber' 'cost' 'Washington' 'mirror']
 ['torch' 'crash' 'brand' 'line' 'elm']
 ['cemetery' 'billboard' 'wood' 'English' 'minute']
 ['Venus' 'toothbrush' 'seal' 'whisk' 'shovel']]
[[ 2  2  1  1  0]
 [ 1  0  1  0  0]
 [ 2  1  1  1  1]
 [ 1  0  0  2  2]
 [ 2  2 -1  0  2]]


In [8]:
import my_model as model

In [9]:
start_time=datetime.now()
while my_game.game_completed == False:
    # here we ignore that team 1 and team 2 will have different models and assume that regardless 
    # of which team's turn it is, we use the same model for the spymaster and the operative
    
    if my_game.waiting_on == 'spymaster':
        clue_start_time=datetime.now()
        clue_word, clue_count = model.generate_clue(game_id=1,
                                                    team_num=my_game.curr_team, 
                                                    gameboard=my_game.gameboard)
        print(f'Clue generated. Duration = {datetime.now()-clue_start_time}. Total elapsed time = {datetime.now()-start_time}')
        print(f'My bot says: "{clue_word} for {clue_count}!"')
        my_game.clue_given(clue_word, clue_count)
    else:
        guess_start_time=datetime.now()
        guesses = model.generate_guesses(game_id = 1,
                                         team_num = my_game.curr_team, 
                                         clue_word = my_game.curr_clue_word, 
                                         clue_count = my_game.curr_clue_count, 
                                         unguessed_words = my_game.gameboard.unguessed_words(),
                                         boardwords = my_game.gameboard.boardwords, 
                                         boardmarkers = my_game.gameboard.boardmarkers)
        print(f'Guesses generated. Duration = {datetime.now()-guess_start_time}. Total elapsed time = {datetime.now()-start_time}')
        print(f'guesses = {guesses}')
        my_game.guesses_given(guesses)

Clue generated. Duration = 0:00:10.370123. Total elapsed time = 0:00:10.370780
My bot says: "already for 6!"
Guesses generated. Duration = 0:00:00.000095. Total elapsed time = 0:00:10.371254
guesses = ['sick', 'team', 'cost', 'line', 'brand', 'clean']
Clue generated. Duration = 0:00:08.238169. Total elapsed time = 0:00:18.610071
My bot says: "flashlights for 3!"
Guesses generated. Duration = 0:00:00.000092. Total elapsed time = 0:00:18.610515
guesses = ['torch', 'toothbrush', 'shovel']
Clue generated. Duration = 0:00:07.011970. Total elapsed time = 0:00:25.623141
My bot says: "death for 2!"
Guesses generated. Duration = 0:00:00.000048. Total elapsed time = 0:00:25.623479
guesses = ['crash', 'cemetery']
Clue generated. Duration = 0:00:06.644709. Total elapsed time = 0:00:32.268694
My bot says: "mean for 3!"
Guesses generated. Duration = 0:00:00.000054. Total elapsed time = 0:00:32.269021
guesses = ['minute', 'body', 'English']
Clue generated. Duration = 0:00:05.108601. Total elapsed tim

In [10]:
my_game.logger.game_log

{'in_progress': False,
 'events': [{'event': 'clue_given',
   'timestamp': datetime.datetime(2021, 1, 18, 16, 30, 42, 875133),
   'team_num': 1,
   'clue_word': 'already',
   'clue_count': 6,
   'legal_clue': 'Yes'},
  {'event': 'guess made',
   'timestamp': datetime.datetime(2021, 1, 18, 16, 30, 42, 875294),
   'team_num': 1,
   'word_guessed': 'sick',
   'result': 1},
  {'event': 'guess made',
   'timestamp': datetime.datetime(2021, 1, 18, 16, 30, 42, 875360),
   'team_num': 1,
   'word_guessed': 'team',
   'result': 1},
  {'event': 'guess made',
   'timestamp': datetime.datetime(2021, 1, 18, 16, 30, 42, 875416),
   'team_num': 1,
   'word_guessed': 'cost',
   'result': 1},
  {'event': 'guess made',
   'timestamp': datetime.datetime(2021, 1, 18, 16, 30, 42, 875752),
   'team_num': 1,
   'word_guessed': 'line',
   'result': 1},
  {'event': 'guess made',
   'timestamp': datetime.datetime(2021, 1, 18, 16, 30, 42, 875795),
   'team_num': 1,
   'word_guessed': 'brand',
   'result': 1},
  

# TESTING
(Not important past this point)

In [ ]:
import spacy # after installing, be sure to run 'python -m spacy download en_core_web_lg'
import itertools
import pickle
import os


nlp = spacy.load("en_core_web_lg")
# old_clue_word_distances = pickle.load(open('clue_word_distances.pkl','rb'))
clue_word_distances = pickle.load(open(os.path.join('distances', 'clue_word_distances.pkl'),'rb'))
# clue_word_distances_extra = pickle.load(open('extra_test_clue_word_distances.pkl','rb'))
# spacy_one_clue_word_distances = pickle.load(open('spacy_one_clue_word_distances.pkl','rb'))
# spacy_two_clue_word_distances = pickle.load(open('spacy_two_clue_word_distances.pkl','rb'))
# spacy_three_clue_word_distances = pickle.load(open('spacy_three_clue_word_distances.pkl','rb'))

def distfn(word1, word2):
    tokens = nlp(word1 + " " + word2)
    return 1 - tokens[0].similarity(tokens[1])

In [ ]:
len(clue_word_distances["distances"][0])

In [ ]:
np.array(clue_word_distances["distances"]).shape

In [ ]:
len(clue_word_distances["clue_words"])

In [ ]:
distfn('shield','sugar')

In [ ]:
pickle.load(open(os.path.join('prev_games','101736.pkl'),'rb'))